In [ ]:
# author: Vladimir Osin 

In [ ]:
import os
import torch
import torch.nn            as nn
import torch.optim         as optim
import torch.nn.functional as F
import mlflow              as mf
import mlflow.pytorch
import shutil

from torch.utils.data import DataLoader
from torchvision      import datasets, transforms
from torch.autograd   import Variable

## Task: Classification of Kuzushiji-MNIST numbers

### A bit of History 
Recorded historical documents give us a peek into the past. We are able to glimpse the world before our time; and see its culture, norms, and values to reflect on our own. Japan has very unique historical pathway. Historically, Japan and its culture was relatively isolated from the West, until the Meiji restoration in 1868 where Japanese leaders reformed its education system to modernize its culture. 

This caused drastic changes in the Japanese language, writing and printing systems. Due to the modernization of Japanese language in this era, cursive Kuzushiji (くずし字) script is no longer taught in the official school curriculum. Even though Kuzushiji had been used for over 1000 years, most Japanese natives today cannot read books written or published over 150 years ago.

The result is that there are hundreds of thousands of Kuzushiji texts that have been digitised but have never been transcribed, and can only currently be read by a few experts. We've built Kuzushiji-MNIST and sister datasets by taking handwritten characters from these texts and preprocessing them in a format similar to the MNIST dataset, to create easy to use benchmark datasets that are more modern and difficult to classify than the original MNIST dataset.

[Kuzushiji-MNIST dataset on Kaggle](https://www.kaggle.com/anokas/kuzushiji)

[Paper: Deep Learning for Classical Japanese Literature](https://arxiv.org/pdf/1812.01718.pdf)

### Task description 

- Build hand-written characters recognition system using convolutional neural networks
- Use MLflow for tracking experimetns 

![](https://raw.githubusercontent.com/rois-codh/kmnist/master/images/kmnist_examples.png)

### Parameters

In [ ]:
# our parameters
args = {'batch_size'      : 10,      # input batch size for training
        'test_batch_size' : 100,     # input batch size for testing
        'epochs'          : 1,      # number of epochs to train
        'seed'            : 42,      # fixing seed for reproducibility
        'learning_rate'   : 0.03,    # learning rate for optimizer
        'momentum'        : 0.1}     # momentum for optimizer

# GPU or CPU training based on your device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('your device is {0}'.format(device))

### Neural network architecture (CNN)

In [ ]:
class Awesome_Net(nn.Module):
    def __init__(self):
        super(Awesome_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10,  kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

### Dataset and Dataloader

In [ ]:
# preparing dataset
transform     = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.KMNIST('../data', train=True,  download=True, transform=transform)
test_dataset  = datasets.KMNIST('../data', train=False, download=True, transform=transform)

In [ ]:
# preparing loader
train_loader = DataLoader(train_dataset, batch_size=args['batch_size'], shuffle=True, num_workers=1)
test_loader  = DataLoader(test_dataset,  batch_size=args['test_batch_size'], shuffle=True, num_workers=1)


### Model and Optimizer

In [ ]:
model     = Awesome_Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args['learning_rate'], momentum=args['momentum'])

### Training and Testing functions

In [ ]:
def train(epoch):
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 1000  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))
            step = epoch * len(train_loader) + batch_idx
            mf.log_metric('train_loss',  loss.data.item())

            
def test(epoch):
    model.eval()
    test_loss = 0
    correct   = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output       = model(data)
            test_loss   += F.nll_loss(output, target, reduction='sum').data.item() # sum up batch loss
            pred         = output.data.max(1)[1] # get the index of the max log-probability
            correct     += pred.eq(target.data).cpu().sum().item()

    test_loss    /= len(test_loader.dataset)
    test_accuracy = 100.0 * correct / len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), test_accuracy))
        
    mf.log_metric('test_loss',     test_loss)
    mf.log_metric('test_accuracy', test_accuracy)

### MLflow Experiment

In [ ]:
# let's connect to the tracking server 
mf.set_tracking_uri('http://127.0.0.1:5000')

In [ ]:
# let's connect to the tracking server 
experiment_id = mf.set_experiment("Tracking - ML Task")

In [ ]:
torch.manual_seed(args["seed"])

with mf.start_run():
    
    # creating a temporary folder 
    output_dir   = 'tmp/'
    os.makedirs(output_dir, exist_ok=True)
    
    # logging parameters
    mf.log_params(args)

    # perform training and testing
    for epoch in range(1, args['epochs'] + 1):
        train(epoch)
        test(epoch)
    
    # store trained model
    mf.pytorch.log_model(model, "model")
        
    # logging all model artifacts 
    mf.log_artifacts(output_dir)
    
    # remove temporary folder
    shutil.rmtree(output_dir)